Problem:

Task 1 less than k

Given a positive integer “k” and a list of integer numbers, look for the numbers within the list, that are less than k. Consider an appropriate number of qubits and explain why your proposal is valid for all kinds of numbers in case 

```
def less_than_k (int:k, list[int] ,list_n):
     """"""
     k : integer value that is the positive number to compare in list_n,
     list_n : integer list that has positive numbers.
     Return the numbers that are in list_n and are less than k 
     """
          # use a framework that works with quantum circuits, qiskit, cirq, pennylane, etc. 
          # consider print your quantum circuit,
```

Example:

A = less_than_k (7,[4,9,11,14,1,13,6,15])
print(A)

“4,1,6”

Terminology:

Query:  Values given in the list

Key:    Values to be searched in the list

### Importing libraries

Framework Used: Qiskit

In [2]:
import qiskit
qiskit.__version__

'1.0.2'

In [3]:
import qiskit_ibm_runtime
qiskit_ibm_runtime.__version__

'0.22.0'

In [4]:
import math
import numpy as np
from random import sample, randint

In [5]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
from qiskit.circuit.library import GroverOperator, MCMT, ZGate
from qiskit.visualization import plot_distribution
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

In [6]:
from qiskit_ibm_runtime.fake_provider import FakeCairoV2
from qiskit_ibm_runtime import SamplerV2 as Sampler

In [7]:
# init a Fake backend for using a simulated backend for transpilation and running
backend = FakeCairoV2()
backend.name

'fake_cairo'

### Sample Problem Generator

In [8]:
def random_test_gen(max_limit, num_samples):
    """Generates a list of random integers in the range(0, max_limit)
    with 'num_samples' elements in the list

    Args:
        max_limit (int): maximum possible number in the list
        num_samples (int): number of samples to be generated

    Returns:
        test_list (list): list of random integers < max_limit
        max_bits (int): number of qubits required in the circuit 
    """
    # random sampling 
    test_list = sample(range(1, max_limit), num_samples)
    
    # find the maximum element in the list
    max_sample = max(test_list)
    
    # get the number of bits for representing max_in_test
    num_bits = len( bin(max_sample)[2:] )
    # get the range of solution space (0, 2**num_bits)
    max_range = 2**num_bits-1
    max_bits = len( bin(max_range)[2:] )
    
    assert num_bits == max_bits, 'error, num_bits and max_bits are different'
    
    return test_list, max_bits

### Helper functions

In [9]:
def generate_binary_states(test_list, max_bits):
    """Generate list of binary states corresponding to the list of integer states

    Args:
        test_list (list): list of random integers < max_limit
        max_bits (int): number of qubits required in the circuit

    Returns:
        bin_states: list of corresponding binary states
    """
    bin_states = []
    for sample_i in test_list:
        # get bin representation of int and store in list
        bin_key = bin(sample_i)[2:]
        # left-pad with 0 maintains the same number of qubits for all inputs
        padded_key = "0"*(max_bits-len(bin_key)) + bin_key
        
        bin_states.append(padded_key)
    
    return bin_states

### State preparation

In [10]:
def generate_query_state_vector_from_input(query_list, N):
    """Initialize the system in equal superposition of the desired 
    query states given in the input list 

    Args:
        query_list (list): list of positive integers
        N (int): total possible states in the system

    Returns:
        sv: Statevector in superposition of the desired query states
    """
    init_state_amps = np.zeros(N)
    # assign equal probability amplitude to all input query states
    prob_ampl = np.sqrt(1/len(query_list))
    
    # generate the Statevector using respective state amplitudes 
    for q_st in query_list:
        init_state_amps[q_st] = prob_ampl
        
    sv = Statevector(init_state_amps)
    
    return sv

### Grover Oracle

In [11]:
def grover_oracle(key_states):
    """Generate a Grover oracle for (multiple) key states 
    (where keys are the values to be searched in the input list)

    Args:
        key_states (list): list of positive integers to be searched in the input
        
    Returns:
        oracle: Grover oracle to search for key states
    """
    num_qubits = len(key_states[0])
    
    oracle = QuantumCircuit(num_qubits)
    
    for key in key_states:
        # Qiskit bit ordering
        reversed_key = key[::-1]
        
        # find indices of all the '0' bits in the key
        zero_indices = [idx for idx in range(num_qubits) if reversed_key.startswith("0", idx)]
        if zero_indices == []:
            continue
        
        # add multi controlled Z-gate sandwiched between X-gates 
        oracle.x(zero_indices)
        oracle.compose(MCMT(ZGate(), num_qubits-1, 1), inplace=True)
        oracle.x(zero_indices)
        
    return oracle

### Merged flow

In [12]:
def less_than_k (k, list_n, max_bits):
    """
    Args:
        k (int): integer value that is the positive number to compare in list_n
        list_n (list): integer list that has positive numbers
        max_bits (int): Max number of qubits used
        
    Returns:
        result (list): Return the numbers that are in list_n and are less than k 
    """

    query_list = list_n
    
    # Grover search only gives advantage if number of keys to be searched
    # is less than half the number of input queries due to amplitude amplification

    # Generate key_list of positive integers to be searched
    # set flag if k exceeds the limit
    # if flip_flag is True, 
    # then select the minimum instead of maximum probabilities to get the solution
    flip_flag = 0

    if (k-1) >= ((2**max_bits)/2):
        print('num of Key is greater than half the num of Queries')
        flip_flag = 1
        key_list = [i for i in range(k, 2**max_bits)]
    else:
        key_list = [i for i in range(0, k)]
        
        
    # generate binary representation of positive numbers in the list

    query_states = generate_binary_states(query_list, max_bits)
    print("Query states: \n", query_states)

    key_states = generate_binary_states(key_list, max_bits)
    print("Key states: \n", key_states)


    N = 2** max_bits
    # State preparation
    query_state_init = generate_query_state_vector_from_input(query_list, N)
    
    # Generate Grover oracle
    # key_states = ["011", "100"]
    oracle = grover_oracle(key_states)
    oracle.draw(output="mpl", style="iqp")
    
    # compose gover operator
    grover_op = GroverOperator(oracle)
    grover_op.decompose().draw(output="mpl", style="iqp")

    # estimate number of iterations
    optimal_num_iter = math.floor(
        math.pi / (4* math.asin( (len(key_states) / 2**grover_op.num_qubits) **0.5 ))
    )


    qc = QuantumCircuit(grover_op.num_qubits)

    qc.prepare_state(query_state_init)

    qc.compose(grover_op.power(optimal_num_iter), inplace=True)

    qc.measure_all()
    qc.draw(output="mpl", style="iqp")

        
    target = backend.target
    pm = generate_preset_pass_manager(target=target, optimization_level=3)

    circuit_isa = pm.run(qc)
    # circuit_isa.draw(output="mpl", idle_wires=False, style="idp")

    sampler = Sampler(backend=backend)
    sampler.options.default_shots = 10_000
    result = sampler.run([circuit_isa]).result()
    dist = result[0].data.meas.get_counts()

    sorted_counts_by_indx = np.zeros(N)
    for key, val in dist.items():
        # print(key, val)
        int_idx =int(key, 2)
        # print(int(key, 2))
        sorted_counts_by_indx[int_idx] = val
    sorted_counts_by_indx

    mean_of_distrib = np.percentile(sorted_counts_by_indx, 85)

    final_result = np.where(sorted_counts_by_indx >= mean_of_distrib)
    final_result = final_result #[0] 
    final_result #, mean_of_distrib #, prob_distrib
    print(final_result, mean_of_distrib)
    print(sorted(query_list))
    print(key_list)

    plot_distribution(dist)
    
    return final_result

### Main

In [13]:
# max possible integer in the list
# N = randint(2, 100)
N = 16 


# number of samples to generated in the list
# num_samples = randint(2,N)
num_samples = 8  


# generate query_list of positive integers for the problem statement
query_list, max_bits = random_test_gen(N, num_samples)
print("Query list: \n", query_list)
print("Max qubits: ", max_bits)

# set the k value of the problem
# k = randint(2, max(query_list)-1)
k = 6 
print("K: ", k)


Query list: 
 [3, 9, 10, 12, 5, 6, 7, 2]
Max qubits:  4
K:  6


In [15]:
result = less_than_k(k, query_list, max_bits)

Query states: 
 ['0011', '1001', '1010', '1100', '0101', '0110', '0111', '0010']
Key states: 
 ['0000', '0001', '0010', '0011', '0100', '0101']
(array([2, 3, 5], dtype=int64),) 738.75
[2, 3, 5, 6, 7, 9, 10, 12]
[0, 1, 2, 3, 4, 5]


References:

[1] Deutsch, David, and Richard Jozsa. "Rapid solution of problems by quantum computation." Proceedings of the Royal Society of London. Series A: Mathematical and Physical Sciences 439.1907 (1992): 553-558.

[2] Bernstein, Ethan, and Umesh Vazirani. "Quantum complexity theory." SIAM Journal on computing 26.5 (1997): 1411-1473.

[3] Grover, Lov K. , "A fast quantum mechanical algorithm for database search", Proceedings of the 28th Annual ACM Symposium on the Theory of Computing (1996), arXiv:quant-ph/9605043

https://qiskit-community.github.io/qiskit-algorithms/tutorials/06_grover.html#State-preparation